# Dependencies

In [25]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

# Creating/Cleaning Training Data

In [26]:
# training_df = pd.read_csv('training_data.csv', index_col=0)
training_df = pd.read_csv('training_data_v2.csv', index_col=0)

training_df['Analyst Rating'] = ""

buy_classification = ['buy', 'outperform', 'positive']

for index, row in training_df.iterrows():
    ticker = row['Ticker']
    stock_df = pd.read_csv('../Data_Collection/Analyst_Ratings/' + ticker + '.csv')
    classification_dict = {"Buy": 0, "Don't Buy": 0}
    
    for item in stock_df['Rating']:
        intersection = list(set(str(item).lower().split(' ')) & set(buy_classification))
        if len(intersection) > 0:
            classification_dict["Buy"] += 1
        else:
            classification_dict["Don't Buy"] += 1
            
    values = list(classification_dict.values())
    keys = list(classification_dict.keys())
    if keys[values.index(max(values))] == "Buy":
        training_df.at[index, 'Analyst Rating'] = 1
    else:
        training_df.at[index, 'Analyst Rating'] = 0

# Scaling the data
Scaler = MinMaxScaler()
cols = [x for x in training_df.columns if x not in ['Analyst Rating', 'Ticker']]
training_df[cols] = Scaler.fit_transform(training_df[cols])
training_df = training_df.dropna(axis=0)

training_df

,Ticker,Conversations,Headlines,liquidityRatio,ROE,shortRatio,Analyst Rating
0,NVDA,0.424561,0.892070,0.772970,0.330949,0.000000,1
1,MSFT,0.729825,0.378855,0.415502,0.393405,0.521978,1
2,TSLA,0.533333,0.555066,0.249411,0.188470,0.225275,1
3,AAPL,0.800000,0.665198,0.081106,1.000000,0.115385,1
4,GOOGL,0.445614,0.389868,0.531034,0.296673,0.670330,1
5,FB,0.287719,0.387665,1.000000,0.333259,0.401099,1
6,JNJ,0.473684,0.801762,0.091215,0.217923,0.813187,1
7,VZ,0.385965,0.563877,0.131575,0.295145,0.785714,0
8,T,0.638596,0.244493,0.000000,0.000000,0.906593,0
10,CSCO,0.031579,0.325991,0.186831,0.290197,0.978022,1


# Developing the model

In [40]:
LR = LogisticRegression()

training_cols = [x for x in training_df.columns if x not in ['Analyst Rating', 'Ticker']]
X_train, X_test, y_train, y_test = train_test_split(training_df[training_cols], 
                                                    training_df['Analyst Rating'], test_size=0.33, random_state=42)

y_train = y_train.astype('int')
y_test = y_test.astype('int')

LR = LR.fit(X_train,y_train)
y_predictions = LR.predict(X_test)

print(classification_report(y_test, y_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.86      1.00      0.92         6

    accuracy                           0.86         7
   macro avg       0.43      0.50      0.46         7
weighted avg       0.73      0.86      0.79         7



/Users/venkyramaraju/Downloads/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
